In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.integrate
import tidynamics
import threefry

In [ ]:
def random_sign(n):
    return (-1+2*np.random.randint(0, 2, size=n)).tolist()

def random_flip(n, proba):
    result = np.ones(n, dtype=int)
    result[np.random.random(size=n)<proba] = -1
    return result

def compute_rho(indices, Nx):
    result = np.zeros(Nx, dtype=int)
    for idx in indices:
        result[idx%Nx] += 1
    return result


In [ ]:
r = threefry.rng(98765)

In [ ]:
# Test active Fischer KPP


N = 2000

Nx = 2000

ρ_max = 10

In [ ]:
many_rho = []

In [ ]:
k1 = 0.1 # rate for A->2A
k2 = 0.01 # rate for 2A -> nil

p_flip = 1/4

p_idx = np.random.randint(0, Nx//8, size=N)
p_vel = np.array(random_sign(N))

plt.figure()

new_rho = []
N_total = []
for i in range(2000):
    p_idx += p_vel
    p_vel *= random_flip(len(p_vel), p_flip)
    new_p = []
    kill_p = []
    ρ = compute_rho(p_idx, Nx)
    for j in range(Nx):
        count = ρ[j]
        if count==0:
            continue
        proba_1 = count*k1/ρ_max
        if count>1:
            proba_2 = count*(count-1)*k2/ρ_max**2 # Rohlf Eq. (4)
        else:
            proba_2 = 0
        xi = np.random.random()
        reaction = 0
        if proba_1+proba_2 > xi:
            xi = np.random.random()*(proba_1+proba_2)
            if xi < proba_1:
                reaction = 1
            else:
                reaction = 2

        if reaction==1:
            # create a particle
            new_p.append(j)
        elif reaction==2:
            # destroy two particles
            kill_p.append(j)
            kill_p.append(j)
    p_idx = p_idx.tolist()
    p_vel = p_vel.tolist()
    kill_indices = []
    for k in kill_p:
        for l, idx in enumerate(p_idx):
            if idx==k and l not in kill_indices:
                kill_indices.append(l)
                continue
    kill_indices.sort()
    kill_indices.reverse()
    for k in kill_indices:
        p_idx.pop(k)
        p_vel.pop(k)
    p_idx = np.array([*p_idx, *new_p])
    p_vel = np.array([*p_vel, *random_sign(len(new_p))])
    N_total.append(len(p_idx))
    ρ = compute_rho(p_idx, Nx)
    if i%200==0:
        plt.plot(ρ)
    new_rho.append(ρ)
many_rho.append(new_rho.copy())

In [ ]:
plt.figure()
plt.plot(N_total)

In [ ]:
all_rho = np.array(many_rho)

In [ ]:
all_rho.shape

In [ ]:
plt.figure()
for i in range(0, 20, 2):
    plt.plot(all_rho[:,i*100,:].mean(axis=0))


In [ ]:

def vv(n, p_f):
    p_nf = 1-p_f
    result = 0
    for i in range(n+1):
        result += p_nf**i * p_f**(n-i) * (-1)**(n-i) * scipy.special.comb(n, i)
    return result


In [ ]:
[vv(i, 0.1) for i in range(10)]

In [ ]:
p_f = 0.01
D = 0.5 + 1/(np.exp(2*p_f)-1)

xx_data = []
vv_data = []
for hop in range(256):
    x = 0
    v = 1

    v_data = [x]
    x_data = [v]
    for i in range(200000):
        x += v
        if r.random_uniform() < p_f:
            v *= -1
        v_data.append(v)
        x_data.append(x)

    vv_data.append(np.array(v_data))
    xx_data.append(np.array(x_data))


In [ ]:
N_steps = 5000
steps = np.arange(N_steps)
vacf = tidynamics.acf(v_data)[:N_steps]
msd = tidynamics.msd(x_data)[:N_steps]


In [ ]:
all_vacf = np.array([tidynamics.acf(v_data)[:N_steps] for v_data in vv_data])

In [ ]:
plt.figure()
m = all_vacf.mean(axis=0)
s = all_vacf.std(axis=0)

plt.plot(m)
plt.plot(m+s, ls='--')
plt.plot(m-s, ls='--')


In [ ]:
plt.figure()
plt.axhline(D)
plt.plot(np.cumsum(all_vacf.mean(axis=0)))

In [ ]:
all_msd = np.array([tidynamics.msd(x_data)[:N_steps] for x_data in xx_data])

In [ ]:
plt.figure()

m = all_msd.mean(axis=0)[:N_steps]

plt.plot(steps, 2*D*steps, label='th.')

fit = np.polyfit(steps, m, 1)

plt.plot(steps, np.poly1d(fit)(steps), label='fit')

plt.plot(steps, m, label='simu.')


plt.legend()

In [ ]:
fit[0]/2

In [ ]:
0.5 + 1/(np.exp(2*p_f) - 1), 1/(2*p_f)